<a href="https://colab.research.google.com/github/JosenildoVicente/projetoInteligenciaArtificial/blob/main/projeto_inteligencia_artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance
import statistics
import math
import time

# Download dos dados

In [2]:
data = {}
data['both_test_unbalanced'] = pd.read_csv( 'https://raw.githubusercontent.com/JosenildoVicente/projetoInteligenciaArtificial/main/dataset/Both%20(complete)/both_test_unbalanced.csv', encoding="ISO-8859-1" )
data['both_test_balanced'] = pd.read_csv( 'https://raw.githubusercontent.com/JosenildoVicente/projetoInteligenciaArtificial/main/dataset/Both%20(complete)/both_test_balanced.csv', encoding="ISO-8859-1" )
data['pcr_unbalanced'] = pd.read_csv( 'https://raw.githubusercontent.com/JosenildoVicente/projetoInteligenciaArtificial/main/dataset/RT-PCR/pcr_unbalanced.csv', encoding="ISO-8859-1" )
data['pcr_balanced'] = pd.read_csv( 'https://raw.githubusercontent.com/JosenildoVicente/projetoInteligenciaArtificial/main/dataset/RT-PCR/pcr_balanced.csv', encoding="ISO-8859-1" )
data['rapid_unbalanced'] = pd.read_csv( 'https://raw.githubusercontent.com/JosenildoVicente/projetoInteligenciaArtificial/main/dataset/Rapid/rapid_unbalanced.csv', encoding="ISO-8859-1" )
data['rapid_balanced'] = pd.read_csv( 'https://raw.githubusercontent.com/JosenildoVicente/projetoInteligenciaArtificial/main/dataset/Rapid/rapid_balanced.csv', encoding="ISO-8859-1" )

In [ ]:
data['pcr_balanced']

#Algoritmos

##Criação do Decision Tree

In [ ]:
def my_decisionTree():
  pass

##Criação do algoritmo KNN

In [4]:
def calc_neighbors(k,x_train,x_test,sample):
  dists = {}
  ind = 0
  for sample_train in x_train.index:
    dist = distance.euclidean(x_train.loc[sample_train],x_test.loc[sample])
    dists[ind] = [dist, sample_train]
    ind+=1
  k_neigh = sorted(dists, key= dists.get)[:k]
  index_neigh = []
  for j in k_neigh:
    index_neigh.append(dists[j][1])
  return index_neigh

In [5]:
def calc_class(sample_neighbors,y_train):
  out = []

  for i in sample_neighbors:
    out.append(y_train[i])

  try:  
    result = statistics.mode(out)
  except:
    result = out[0]

  return result

In [6]:
def my_knn(k,x_train,x_test,y_train):
  time_start = time.time()

  y_model = []

  for sample in x_test.index:
    sample_neighbors = calc_neighbors(k,x_train,x_test,sample)

    sample_class = calc_class(sample_neighbors,y_train)

    y_model.append(sample_class)

  time_total = time.time() - time_start 

  return [y_model,time_total]

##Criação do Random Forests

In [7]:
def my_randomForests(x_train,x_test,y_train):
  pass

##Criação do SVM

In [8]:
def my_svm():
  pass

#Rodar algoritmos

In [9]:
x_treino, x_teste, y_treino, y_teste = train_test_split(data['pcr_balanced'].drop('Class',axis=1),data['pcr_balanced']['Class'],test_size=0.25, stratify=data['pcr_balanced']['Class'])

In [ ]:
x_treino

In [11]:
x_treino.loc[x_treino.index[0]]

Symptom- Throat Pain              1
Symptom- Dyspnea                  1
Symptom- Fever                    1
Symptom- Cough                    0
Symptom- Headache                 1
Symptom- Taste Disorders          1
Symptom- Olfactory Disorders      1
Symptom- Coryza                   1
Gender                            1
Are you a health professional?    1
Name: 1487, dtype: int64

In [17]:
resultado,tempo = my_knn(5,x_treino,x_teste,y_treino)

In [13]:
result_knn_3 = resultado

In [22]:
tempo

170.9317581653595

In [18]:
accuracy_score(y_teste, resultado)

0.9279475982532751

#Ignorar os de baixo


In [ ]:
# data[data['Symptom- Coryza']==1][data['Symptom- Headache']==1][data['Symptom- Cough']==1][data['Symptom- Taste Disorders']==1][data['Symptom- Olfactory Disorders']==1][data['Symptom- Throat Pain'] == 1][data['Symptom- Dyspnea'] == 1][data['Symptom- Fever']==1]

In [ ]:
# x_treino, x_teste, y_treino, y_teste = train_test_split(data.drop('Class',axis=1),data['Class'],test_size=0.25, stratify=data['Class'])
# print(x_treino.shape,x_teste.shape)

In [ ]:
# K = 5

In [ ]:
# modelo = KNeighborsClassifier(n_neighbors=K, weights='uniform')

In [ ]:
# modelo.fit(x_treino, y_treino)

In [ ]:
# y_modelo = modelo.predict(x_teste)

In [ ]:
# acuracia = accuracy_score(y_modelo, y_teste)

In [ ]:
# acuracia